# DB연동
- DB연동하는 것은 에러가 없어야 함
- 해당 코드는 무조건 돌아가게끔 만들어야 함
- MySQL에 문제가 생김
    + 대처방안 : try-catch 구문

In [6]:
numerator = 10 
denominator = 0

result = numerator / denominator

# 하고 싶은 것은 결과 확인 출력을 꼭 하고 싶음, 어떤 일이 있어도
print("결과 확인")

ZeroDivisionError: division by zero

In [10]:
try : 
    numerator = 10 
    denominator = 0
    result = numerator / denominator
except ZeroDivisionError:
    print("Error: 0으로 나눌 수 없음")
else:
    print("결과확인 : ", result)
finally:
    print("결과 확인")

Error: 0으로 나눌 수 없음
결과 확인


In [14]:
try : 
    numerator = 10 
    denominator = int(input("숫자를 입력하세요!!"))
    result = numerator / denominator
except ZeroDivisionError:
    print("Error: 0으로 나눌 수 없음")
except ValueError:
    print("invalid literal for int() with")
else:
    print("결과확인 : ", result)
finally:
    print("결과 확인")

숫자를 입력하세요!! aaa


invalid literal for int() with
결과 확인


# Step 1. Connect to Server

In [16]:
import os 
from mysql import connector

PASSWORD = '1234'

- 기초문법 중급 레벨 (with vs python decorator)

In [17]:
try:
    with connector.connect(
        host = 'localhost', 
        user = 'root', 
        password = PASSWORD
    ) as database:
        print(f"Database 객체 : {database}")
except connector.Error as e:
    print(e)

Database 객체 : <mysql.connector.connection_cext.CMySQLConnection object at 0x000002A719A779D0>


## Step 2. Creating New DB

In [21]:
try:
    with connector.connect(
        host = 'localhost', 
        user = 'root', 
        password = PASSWORD
    ) as database:
        print(f"Database 객체 : {database}")
        
        with database.cursor() as cursor:
            # 쿼리문 작성
            # create_db = "CREATE DATABASE book_ratings;"
            # cursor.execute(create_db)

            # 생성된 DB 확인 쿼리문 쓰기
            show_existing_db = "SHOW DATABASES;"
            cursor.execute(show_existing_db)
            for db in cursor:
                print(db)

except connector.Error as e:
    print(e)

Database 객체 : <mysql.connector.connection_cext.CMySQLConnection object at 0x000002A719A89790>
('book_ratings',)
('classicmodels',)
('information_schema',)
('instacart',)
('mydata',)
('mysql',)
('performance_schema',)
('sakila',)
('sys',)
('testdb',)
('titanic',)
('world',)


- 특정 DB에 직접 연결하겠습니다! 

In [22]:
try:
    with connector.connect(
        host = 'localhost', 
        user = 'root', 
        password = PASSWORD, 
        database = "book_ratings" # 이것이 기존 코드와의 차이점
    ) as database:
        print(f"Database 객체 : {database}")
except connector.Error as e:
    print(e)

Database 객체 : <mysql.connector.connection_cext.CMySQLConnection object at 0x000002A719A88A10>


#  Step 3. Creating Tables

In [23]:
create_books_table = """
    CREATE TABLE books(
        id INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
        title VARCHAR(100),
        author VARCHAR(100),
        genre VARCHAR(100),
        release_year YEAR(4)
    );
"""

try:
    with connector.connect(
        host = 'localhost', 
        user = 'root', 
        password = PASSWORD, 
        database = "book_ratings" # 이것이 기존 코드와의 차이점
    ) as database:
        print(f"Database 객체 : {database}")

        # cursor object 접근 후, 위 쿼리 실행
        with database.cursor() as cursor:
            cursor.execute(create_books_table)
            database.commit()

            # 테이블 확인 DESCRIBE
            describe_books = "DESCRIBE books;"
            cursor.execute(describe_books)
            book_schema = cursor.fetchall()
            for column in book_schema:
                print(column)
except connector.Error as e:
    print(e)

Database 객체 : <mysql.connector.connection_cext.CMySQLConnection object at 0x000002A719A8A7D0>
('id', 'int', 'NO', 'PRI', None, 'auto_increment')
('title', 'varchar(100)', 'YES', '', None, '')
('author', 'varchar(100)', 'YES', '', None, '')
('genre', 'varchar(100)', 'YES', '', None, '')
('release_year', 'year', 'YES', '', None, '')


# Step 4. Data Insertion
- 하나의 레코드 추가할 때와 다중 레코드 추가할 때 사용하는 메서드 다름

In [24]:
insert_single_record = "INSERT INTO books (id, title, author, genre, release_year)\
    VALUES (%s, %s, %s, %s, %s)"
single_record = (
    "1", "Choose Yourself! Be Happy, Make Millions, Live the Dream", "James Altucher", "self-help", "2013"
    )

In [25]:
try:
    with connector.connect(
        host = 'localhost', 
        user = 'root', 
        password = PASSWORD, 
        database = "book_ratings" # 이것이 기존 코드와의 차이점
    ) as database:
        print(f"Database 객체 : {database}")

        # 커서 생성 후, 쿼리 실행
        with database.cursor() as cursor:
            cursor.execute(insert_single_record, single_record)
            database.commit()
except connector.Error as e:
    print(e)

Database 객체 : <mysql.connector.connection_cext.CMySQLConnection object at 0x000002A719A22450>


- 다중 레코드를 추가하장

In [27]:
insert_multiple_records = "INSERT INTO books (id, title, author, genre, release_year)\
    VALUES (%s, %s, %s, %s, %s)"
multiple_records = [
    (
        "2", 
        "Skip the Line: The 10,000 Experiments Rule and Other Surprising Advice for Reaching Your Goals",
        "James Altucher",
        "self-help",
        "2021"        
    ),
    (
        "3",
        "The Power of No: Because One Little Word Can Bring Health, Abundance, and Happiness",
        "James Altucher",
        "self-help",
        "2014"
    ),
    (
        "4",
        "The 48 Laws of Power",
        "Robert Greene",
        "self-help",
        "1998"
    ),
    (
        "5",
        "Mastery",
        "Robert Greene",
        "self-help",
        "2012"
    ),
    (
        "6",
        "The Art of Seduction",
        "Robert Greene",
        "self-help",
        "2001"
    ),
]


In [28]:
try:
    with connector.connect(
        host = 'localhost', 
        user = 'root', 
        password = PASSWORD, 
        database = "book_ratings" # 이것이 기존 코드와의 차이점
    ) as database:
        print(f"Database 객체 : {database}")

        # 커서 생성 후, 쿼리 실행
        with database.cursor() as cursor:
            # cursor.execute(insert_single_record, single_record)
            cursor.executemany(insert_multiple_records, multiple_records)
            database.commit()
except connector.Error as e:
    print(e)

Database 객체 : <mysql.connector.connection_cext.CMySQLConnection object at 0x000002A719A3B150>


# Step 5. Selecting Data

In [29]:
try:
    with connector.connect(
        host = 'localhost', 
        user = 'root', 
        password = PASSWORD, 
        database = "book_ratings" # 이것이 기존 코드와의 차이점
    ) as database:
        print(f"Database 객체 : {database}")

        # 커서 생성 후, 쿼리 실행
        with database.cursor() as cursor:
            
            # 쿼리문 작성
            select_cols = "SELECT author, release_year FROM books;"
            cursor.execute(select_cols)

            # 데이터 확인
            df = cursor.fetchall()
            for result in df:
                print(result)
            
except connector.Error as e:
    print(e)

Database 객체 : <mysql.connector.connection_cext.CMySQLConnection object at 0x000002A719A4CCD0>
('James Altucher', 2013)
('James Altucher', 2021)
('James Altucher', 2014)
('Robert Greene', 1998)
('Robert Greene', 2012)
('Robert Greene', 2001)


# SELECT 사용자 정의 함수 만들기
- 어떤 테이블을 조회해도 깔끔하게 데이터프레임으로 만드는 함수를 만드시고,
- 테스트를 해주세요

In [31]:
import pandas as pd

# Function to fetch data from database and return as pandas DataFrame
def fetch_data_to_dataframe(query, database=None):
    try: 
        # Connect to existing database
        with connector.connect(
            host="localhost",
            user="root",
            password=PASSWORD,
            database=database
        ) as existing_database:
            
            with existing_database.cursor() as cursor:
                cursor.execute(query)
                
                column_names = [i[0] for i in cursor.description] # List Comprehension
                
                returned_data = cursor.fetchall()
                
                df = pd.DataFrame(returned_data, columns=column_names)
                return df
    
    except connector.Error as e: 
        print(e)
        return None


query_specific_cols = "SELECT author, release_year FROM books"
df_specific_cols = fetch_data_to_dataframe(query_specific_cols, "book_ratings")
if df_specific_cols is not None:
    print("DataFrame with specific columns:")
    print(df_specific_cols)

DataFrame with specific columns:
           author  release_year
0  James Altucher          2013
1  James Altucher          2021
2  James Altucher          2014
3   Robert Greene          1998
4   Robert Greene          2012
5   Robert Greene          2001


In [33]:
query_all_cols = "SELECT * FROM books"
df_all_cols = fetch_data_to_dataframe(query_all_cols, "book_ratings")
if df_all_cols is not None:
    print("DataFrame with all columns:")
    print(df_all_cols)

DataFrame with all columns:
   id                                              title          author  \
0   1  Choose Yourself! Be Happy, Make Millions, Live...  James Altucher   
1   2  Skip the Line: The 10,000 Experiments Rule and...  James Altucher   
2   3  The Power of No: Because One Little Word Can B...  James Altucher   
3   4                               The 48 Laws of Power   Robert Greene   
4   5                                            Mastery   Robert Greene   
5   6                               The Art of Seduction   Robert Greene   

       genre  release_year  
0  self-help          2013  
1  self-help          2021  
2  self-help          2014  
3  self-help          1998  
4  self-help          2012  
5  self-help          2001  


In [38]:
query_all_cols = "SELECT * FROM dataset2"
DATABASE = "mydata"
df_all_cols = fetch_data_to_dataframe(query_all_cols, DATABASE)
df_all_cols

,ClothingID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,767,33,,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1080,34,,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
...,...,...,...,...,...,...,...,...,...,...
20598,1104,34,Great dress for many occasions,I was very happy to snag this dress at such a ...,5,1,0,General Petite,Dresses,Dresses
20599,862,48,Wish it was made of cotton,"It reminds me of maternity clothes. soft, stre...",3,1,0,General Petite,Tops,Knits
20600,1104,31,"Cute, but see through","This fit well, but the top was very see throug...",3,0,1,General Petite,Dresses,Dresses
20601,1084,28,"Very cute dress, perfect for summer parties an...",I bought this dress for a wedding i have this ...,3,1,2,General,Dresses,Dresses


# Step 6. Update and Delete
- 

In [40]:
update_query = """
    UPDATE 
        books
    SET 
        author = 'Big J'
    WHERE 
        author = 'James Altucher'
"""

try:
    with connector.connect(
        host = 'localhost', 
        user = 'root', 
        password = PASSWORD, 
        database = "book_ratings" # 이것이 기존 코드와의 차이점
    ) as database:
        print(f"Database 객체 : {database}")

        # 커서 생성 후, 쿼리 실행
        with database.cursor() as cursor:
            cursor.execute(update_query)
            database.commit()

            check_query = "SELECT DISTINCT author FROM books"
            cursor.execute(check_query)

            for result in cursor.fetchall():
                print(result)
            
except connector.Error as e:
    print(e)

Database 객체 : <mysql.connector.connection_cext.CMySQLConnection object at 0x000002A719426510>
('Big J',)
('Robert Greene',)


# Step 7. Delete Records

In [42]:
try:
    with connector.connect(
        host = 'localhost', 
        user = 'root', 
        password = PASSWORD, 
        database = "book_ratings" # 이것이 기존 코드와의 차이점
    ) as database:
        print(f"Database 객체 : {database}")

        # 커서 생성 후, 쿼리 실행
        with database.cursor() as cursor:

            delete_query = "DELETE FROM books WHERE release_year <= 2010;"
            cursor.execute(delete_query)
            database.commit()

            check_query = "SELECT DISTINCT release_year FROM books;"
            cursor.execute(check_query)
            for result in cursor.fetchall():
                print(result)
            
except connector.Error as e:
    print(e)

Database 객체 : <mysql.connector.connection_cext.CMySQLConnection object at 0x000002A7199F2750>
(2013,)
(2021,)
(2014,)
(2012,)
